In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests

In [10]:
API_KEY = "8b491b601812f2ddd75d2597"
API_BASE_URL = f"https://v6.exchangerate-api.com/v6/{API_KEY}/latest/USD"
rates = requests.get(API_BASE_URL).json()['conversion_rates']

In [13]:
users = pd.read_csv("./Datasets/users.csv")
purchases = pd.read_csv("./Datasets/purchases.csv")
purchases.head(5)

,user_id,SKU,AddedTime,Price,CurrencyISO
0,6192636,personal_offer_starter_pack,2023-01-31 13:16:55.991756+00:00,2.29,GBP
1,5954105,bundle_pack_1,2023-01-06 19:20:33.631714+00:00,1.79,EUR
2,5954105,coin_pack_1,2023-01-07 15:56:47.792655+00:00,0.79,EUR
3,5903715,bundle_pack_1,2023-01-01 18:48:38.391356+00:00,1.99,USD
4,5984323,golden_ticket_season_pass,2023-01-11 21:13:47.161073+00:00,9.99,USD


# Preprocessing

## Normalizing Prices to USD

In [29]:
purchases["PriceUSD"] = purchases.apply(lambda row : row["Price"] / rates[row["CurrencyISO"]], axis=1)
purchases.head(5)

,user_id,SKU,AddedTime,Price,CurrencyISO,PriceUSD
0,6192636,personal_offer_starter_pack,2023-01-31 13:16:55.991756+00:00,2.29,GBP,2.760699
1,5954105,bundle_pack_1,2023-01-06 19:20:33.631714+00:00,1.79,EUR,1.891978
2,5954105,coin_pack_1,2023-01-07 15:56:47.792655+00:00,0.79,EUR,0.835007
3,5903715,bundle_pack_1,2023-01-01 18:48:38.391356+00:00,1.99,USD,1.990000
4,5984323,golden_ticket_season_pass,2023-01-11 21:13:47.161073+00:00,9.99,USD,9.990000


## Join Tables

In [30]:
df = pd.merge(users,purchases,on="user_id")

In [31]:
df

,user_id,registered_time,country_code,SKU,AddedTime,Price,CurrencyISO,PriceUSD
0,6192636,2023-01-30 16:12:02.731706+01:00,GB,personal_offer_starter_pack,2023-01-31 13:16:55.991756+00:00,2.29,GBP,2.760699
1,5954105,2023-01-05 20:33:54.584158+01:00,BE,bundle_pack_1,2023-01-06 19:20:33.631714+00:00,1.79,EUR,1.891978
2,5954105,2023-01-05 20:33:54.584158+01:00,BE,coin_pack_1,2023-01-07 15:56:47.792655+00:00,0.79,EUR,0.835007
3,5903715,2022-12-30 18:47:41.108888+01:00,US,bundle_pack_1,2023-01-01 18:48:38.391356+00:00,1.99,USD,1.990000
4,5984323,2023-01-09 18:57:06.353154+01:00,US,golden_ticket_season_pass,2023-01-11 21:13:47.161073+00:00,9.99,USD,9.990000
...,...,...,...,...,...,...,...,...
1399,5964679,2023-01-07 00:50:45.364926+01:00,US,bundle_pack_1,2023-02-01 13:13:43.680081+00:00,1.49,USD,1.490000
1400,5964679,2023-01-07 00:50:45.364926+01:00,US,coin_pack_1,2023-01-19 13:51:50.503887+00:00,0.99,USD,0.990000
1401,5964679,2023-01-07 00:50:45.364926+01:00,US,bundle_pack_1,2023-02-04 23:15:25.865344+00:00,1.49,USD,1.490000
1402,5964679,2023-01-07 00:50:45.364926+01:00,US,coin_pack_1,2023-01-20 15:43:32.899227+00:00,0.99,USD,0.990000
